# Data Collection Notebook

## Objectives
- Fetch data from Kaggle
- Save it as raw data
- Inspect the data
- Save it under outputs/datasets/collection

## Inputs
- Kaggle JSON file - the authentication token

## Outputs 
- Generate dataset: outputs/datasets/collection/HousingMarket.csv

---

## Change working directory
Changing current working directory to its parent folder

In [1]:
import os 
cwd = os.getcwd()
cwd

'/workspaces/heritage-housing/jupyter_notebooks'

In [2]:
os.chdir(os.path.dirname(cwd))
print("You set a new current working directory")

You set a new current working directory


In [4]:
cwd = os.getcwd()
cwd

'/workspaces/heritage-housing'

## Fetch data from Kaggle
Install Kaggle package to fetch data

In [5]:
%pip install kaggle==1.5.12

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73076 sha256=a84203590e41974909dc05e21e99dcd62bbd1576ee65821a4876c5fa01c6c6d9
  Stored in directory: /home/codespace/.cache/pip/wheels/f5/69/4d/d701fc604b9fb09be59718b4056fd5556a22588ce1f25dd090
Successfully built kaggle
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [kaggle]
Note: you may need to restart the kernel to use updated packages.


Recognise token

In [6]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Define Kaggle dataset and destination folder

In [7]:
KaggleDatasetPath = "codeinstitute/housing-prices-data"
DestinationFolder = "inputs/datasets/raw"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

  0%|                                               | 0.00/49.6k [00:00<?, ?B/s]
100%|███████████████████████████████████████| 49.6k/49.6k [00:00<00:00, 602kB/s]


Unzip downloaded file, delete the zip file, delete the kaggle.json file

In [8]:
! unzip {DestinationFolder}/*.zip -d {DestinationFolder} \
  && rm {DestinationFolder}/*.zip \
  && rm kaggle.json

Archive:  inputs/datasets/raw/housing-prices-data.zip
  inflating: inputs/datasets/raw/house-metadata.txt  
  inflating: inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/house_prices_records.csv  
  inflating: inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/inherited_houses.csv  
